In [1]:
import pandas as pd
import re

In [5]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_Nabil.xlsx",skiprows=1)
# soa_statement_df = pd.read_excel('D:/NPS/Nabil Bank 23 July.xlsx', sheet_name='SOA')

In [6]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,98763101008,-8328.0,D,NEPALREMIT#Fund TransferR77664939219,0.0,NPS-IF-3142849,08/29/2023 00:00:00,NPS-IF-3142849
1,2,98763101008,-25373.0,D,IPAY215157221,0.0,FTMS-181987,08/29/2023 00:00:00,FTMS-181987
2,3,98763101008,-100000.0,D,SMR#Fund Transfer33TF039994239/SMSR148992,0.0,NPS-IF-3142829,08/29/2023 00:00:00,NPS-IF-3142829
3,4,98763101008,-6609.0,D,SMR#Fund Transfer33TF125749051/SMSR148989,0.0,NPS-IF-3142814,08/29/2023 00:00:00,NPS-IF-3142814
4,5,98763101008,-410000.0,D,NEPALREMIT#Fund Transfer2342364858,0.0,NPS-IF-3142813,08/29/2023 00:00:00,NPS-IF-3142813


In [ ]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (row['Desc1'] , row['Desc3']):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            break
        
        
        elif "10000" in roww:
            if re.findall(r'#10*\d{7}',roww):
                continue
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            
        elif re.findall(r'\d{7}-.*', roww):
            id = re.findall(r'[0-9]{7}', roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            
        elif "eod" in roww:
            amount = abs(float(row["Amount"]))
            
            for indexsoa, soarow in soa_statement_df.iterrows():
                
                if soarow["Transaction Type"] == "BankVoucherEntry" and float(soarow["Amount"]) == amount:
                    print(amount)
                    bank_statement_df.loc[index, 'Transaction ID'] = soarow["Transaction Id"]


In [ ]:
soa_statement_df = pd.read_excel('D:/NPS/Nabil Bank 23 July.xlsx', sheet_name='SOA')

In [57]:
bank_statement_df = bank_statement_df.astype(str)
soa_statement_df = soa_statement_df.astype(str)
bank_statement_df.drop(bank_statement_df.index[-1], inplace=True)
soa_statement_df.drop(soa_statement_df.index[-1], inplace=True)

In [8]:
for index, row in bank_statement_df.iterrows():
    for roww in (row['Desc1'] , row['Desc3']):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            break
        
        
        elif "10000" in roww:
            if re.findall(r'#10*\d{7}',roww):
                continue
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            
        elif re.findall(r'\d{7}-.*', roww):
            id = re.findall(r'[0-9]{7}', roww)
            bank_statement_df.loc[index, 'Transaction ID'] = id[0]
            
        


In [11]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [12]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)  

In [13]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID,Mode
0,1,98763101008,8328.0,D,NEPALREMIT#Fund TransferR77664939219,0.0,NPS-IF-3142849,08/29/2023 00:00:00,NPS-IF-3142849,3142849,DR
1,2,98763101008,25373.0,D,IPAY215157221,0.0,FTMS-181987,08/29/2023 00:00:00,FTMS-181987,181987,DR
2,3,98763101008,100000.0,D,SMR#Fund Transfer33TF039994239/SMSR148992,0.0,NPS-IF-3142829,08/29/2023 00:00:00,NPS-IF-3142829,3142829,DR
3,4,98763101008,6609.0,D,SMR#Fund Transfer33TF125749051/SMSR148989,0.0,NPS-IF-3142814,08/29/2023 00:00:00,NPS-IF-3142814,3142814,DR
4,5,98763101008,410000.0,D,NEPALREMIT#Fund Transfer2342364858,0.0,NPS-IF-3142813,08/29/2023 00:00:00,NPS-IF-3142813,3142813,DR
...,...,...,...,...,...,...,...,...,...,...,...
1152,1153,98763101008,323550.0,D,GMEREMIT#GME-R93535033886/United Kingdom,0.0,NPS-IF-3134605,08/29/2023 00:00:00,NPS-IF-3134605,3134605,DR
1153,1154,98763101008,100000.0,D,NEPALREMIT#Fund TransferR17326562499,0.0,NPS-IF-3134597,08/29/2023 00:00:00,NPS-IF-3134597,3134597,DR
1154,1155,98763101008,149987.0,D,GMEREMIT#GME-5610940441080/UAE,0.0,NPS-IF-3134595,08/29/2023 00:00:00,NPS-IF-3134595,3134595,DR
1155,1156,98763101008,3055.0,D,IMEPAY#IMEPAY-202308290014043083-TRFO-9862276004-,0.0,NPS-IF-3134586,08/29/2023 00:00:00,NPS-IF-3134586,3134586,DR


In [19]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%m/%d/%Y %H:%M:%S").dt.date


In [20]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID,Mode
0,1,98763101008,8328.0,D,NEPALREMIT#Fund TransferR77664939219,0.0,NPS-IF-3142849,2023-08-29,NPS-IF-3142849,3142849,DR
1,2,98763101008,25373.0,D,IPAY215157221,0.0,FTMS-181987,2023-08-29,FTMS-181987,181987,DR
2,3,98763101008,100000.0,D,SMR#Fund Transfer33TF039994239/SMSR148992,0.0,NPS-IF-3142829,2023-08-29,NPS-IF-3142829,3142829,DR
3,4,98763101008,6609.0,D,SMR#Fund Transfer33TF125749051/SMSR148989,0.0,NPS-IF-3142814,2023-08-29,NPS-IF-3142814,3142814,DR
4,5,98763101008,410000.0,D,NEPALREMIT#Fund Transfer2342364858,0.0,NPS-IF-3142813,2023-08-29,NPS-IF-3142813,3142813,DR


In [10]:
bank_statement_df['Transaction ID'].count()

1156

In [51]:
bank_statement_df.loc[20]

S.N                                                            21.0
Ac.Number                                             98763101008.0
Amount                                                      -7000.0
Desc1             THAILI#TID#100002401309 XFR From Sunil Poudel Uti
Desc2                                                           0.0
Desc3                                                NPS-IF-2875408
Date                                            07/23/2023 00:00:00
Number                                               NPS-IF-2875408
Transaction ID                                              2875408
Name: 20, dtype: object

In [59]:
banknotmatch = []
bankcr = 0
bankdr = 0


for index1 , bank_statement in bank_statement_df.iterrows():
    
    if str(bank_statement['Transaction ID']) in (soa_statement_df['Transaction Id'].values):
        # bankdr += abs(float(bank_statement["DEBIT"]))
        # bankcr += abs(float(bank_statement["CREDIT"]))
        continue
        
    banknotmatch.append(bank_statement)
   



(banknotmatch)


[S.N                                                             1.0
 Ac.Number                                             98763101008.0
 Amount                                                     -24990.0
 Desc1             IMEPAY#IMEPAY-202307240012059960-TRFO-9818058629-
 Desc2                                                           0.0
 Desc3                                                NPS-IF-2875526
 Date                                            07/23/2023 00:00:00
 Number                                               NPS-IF-2875526
 Transaction ID                                              2875526
 Name: 0, dtype: object,
 S.N                             944.0
 Ac.Number               98763101008.0
 Amount                       -50000.0
 Desc1                       206186465
 Desc2                             0.0
 Desc3                     FTMS-158802
 Date              07/23/2023 00:00:00
 Number                    FTMS-158802
 Transaction ID                 158802
 Na

In [58]:
soanotmatch = []
soacr = 0
soadr = 0
for indexsoa , soa_statement in soa_statement_df.iterrows():
    if str(soa_statement['Transaction Id']) in (bank_statement_df['Transaction ID'].values):
        continue
    soanotmatch.append(soa_statement)

(soanotmatch)

[]